# Assignment 1 - CIC-1205

## Exercise 2 - Diamond price prediction

Student: Balthazar Paixão


## Code


In [1]:
import numpy as np
import pandas as pd

In [5]:
df_diamonds = pd.read_csv("../class-repo/cic1205/data/diamonds.csv")

In [6]:
df_diamonds.sample(3)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
39387,39388,0.31,Premium,D,VVS1,61.2,59.0,1075,4.38,4.35,2.67
1622,1623,0.71,Ideal,H,VVS1,61.8,56.0,3014,5.70,5.75,3.54
21142,21143,1.57,Ideal,H,SI2,60.7,57.0,9298,7.49,7.54,4.56


In [7]:
df_diamonds = df_diamonds.drop(columns=["Unnamed: 0"])

In [8]:
colnames = [column for column in df_diamonds.columns]
colnames

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z']

## Pré-processamento dos dados


Esta etapa é muito importante, uma má execução desta etapa pode causar problemas no treino.

Aqui, verificamos os valores presentes em colunas que sabemos que se tratam de dados categóricos. Então é necessário pensar se estes dados são _nominais_ (valor que indica categoria, sem levar em consideração um "ranking") ou _ordinais_ (valor que indica categoria, mas que carrega uma relação de ordem com demais valores presentes).

Podemos ver que os dados são ordinais. Podemos ver que há valores para os quais vai haver um corte, cor ou claridade do diamante que vai ser ideal ou não, então isso é modelado na codificação.


In [9]:
print(df_diamonds["cut"].unique())
print(df_diamonds["color"].unique())
print(df_diamonds["clarity"].unique())

['Ideal' 'Premium' 'Good' 'Very Good' 'Fair']
['E' 'I' 'J' 'H' 'F' 'G' 'D']
['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF']


In [10]:
df_diamonds["cut"].replace(
    {"Fair": 1, "Good": 2, "Very Good": 3, "Premium": 4, "Ideal": 5}, inplace=True
)
df_diamonds["color"].replace(
    {"J": 1, "I": 2, "H": 3, "G": 4, "F": 5, "E": 6, "D": 7}, inplace=True
)
df_diamonds["clarity"].replace(
    {"I1": 1, "SI2": 2, "SI1": 3, "VS2": 4,
        "VS1": 5, "VVS2": 6, "VVS1": 7, "IF": 8},
    inplace=True,
)

In [11]:
print(df_diamonds["cut"].unique())
print(df_diamonds["color"].unique())
print(df_diamonds["clarity"].unique())

[5 4 2 3 1]
[6 2 1 3 5 4 7]
[2 3 5 4 6 7 1 8]


In [12]:
df_diamonds.sample(3)

,carat,cut,color,clarity,depth,table,price,x,y,z
22176,1.63,3,3,3,59.8,60.0,10216,7.64,7.68,4.58
14321,1.51,1,1,2,65.1,59.0,5801,7.13,7.09,4.63
917,0.84,5,4,2,62.3,55.0,2879,6.08,6.03,3.77


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
# Divisão entre treino e teste
X = df_diamonds.drop(columns=['price'])
y = df_diamonds['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Normalização das features
É importante normalizarmos as features para equilibrar as suas ordens de grandeza, para os mesmos serem processados na etapa de treino. Isso é feito porque a existência de discrepâncias entres essas orders de grandeza prejudica o ajuste dos modelos em alguns algoritmo de aprendizado.

É também importante entender que o ajuste do *scaler* deve ser feito apenas sobre o conjunto de treino,
para evitar que haja vazamento de dados (fenômeno conhecido como [data leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning)).

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(43152, 9)
(43152,)
(10788, 9)
(10788,)


## Treinamento do modelo